In [80]:
import pandas as pd
import glob
import numpy as np

## Load Datasets

In [68]:
rotten_tomatoes_raw = pd.read_csv('rotten_tomatoes/movie_info.csv')

In [69]:
imdb_dict = dict()
num_rows = 0
for path in glob.glob(f'imdb/*.csv'):
    df = pd.read_csv(path)
    imdb_dict[path] = df
    num_rows += df.shape[0]

In [70]:
imdb_raw = pd.concat(imdb_dict.values(), ignore_index=True)

In [91]:
oscar_raw = pd.read_csv('oscar_award/full_data.csv', sep='\t')

## Data Cleaning

### Rotten Tomatoes Ratings

In [72]:
rotten_tomatoes = rotten_tomatoes_raw.dropna()
rotten_tomatoes['critic_score'] = rotten_tomatoes['critic_score'].apply(lambda x: float(x[:-1])/100)
rotten_tomatoes['audience_score'] = rotten_tomatoes['audience_score'].apply(lambda x: float(x[:-1])/100)

/var/folders/02/pnxb037d1s55wxx4bpzhn2s00000gn/T/ipykernel_69691/2766866854.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rotten_tomatoes['critic_score'] = rotten_tomatoes['critic_score'].apply(lambda x: float(x[:-1])/100)
/var/folders/02/pnxb037d1s55wxx4bpzhn2s00000gn/T/ipykernel_69691/2766866854.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rotten_tomatoes['audience_score'] = rotten_tomatoes['audience_score'].apply(lambda x: float(x[:-1])/100)


In [73]:
def clean_date(date):
    year = date[-4:]
    try:
        return int(year)
    except:
        return None
rotten_tomatoes['release_year'] = rotten_tomatoes['release_date'].apply(clean_date)

/var/folders/02/pnxb037d1s55wxx4bpzhn2s00000gn/T/ipykernel_69691/2047567386.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rotten_tomatoes['release_year'] = rotten_tomatoes['release_date'].apply(clean_date)


In [74]:
rotten_tomatoes = rotten_tomatoes.dropna()
rotten_tomatoes['release_year'] = rotten_tomatoes['release_year'].astype(int)

In [75]:
imdb = imdb_raw.drop_duplicates()

In [77]:
# imdb['year'] = imdb['year'].astype(int)

In [81]:
# Try to convert to numeric, forcing errors to NaN
imdb['year'] = pd.to_numeric(imdb['year'], errors='coerce')

# Optionally, filter to a reasonable range (e.g. 1880–2025)
imdb.loc[~imdb['year'].between(1880, 2025), 'year'] = np.nan

In [86]:
imdb = imdb.dropna(subset=['year'])
imdb['year'] = imdb['year'].astype(int)

In [88]:
imdb_tomatoes = rotten_tomatoes.merge(imdb, left_on=['title', 'release_year'], right_on=['movie_name', 'year'])

In [92]:
oscar = oscar_raw.iloc[:, :12]

In [101]:
imdb_tomatoes_oscar = imdb_tomatoes.merge(oscar, left_on='movie_id', right_on='FilmId')